<a href="https://colab.research.google.com/github/privateJh/Cuda-Study-with-colab/blob/main/CUDA_STUDY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GPU 연결 확인**

In [1]:
!pwd

/content


In [2]:
!ls

sample_data


In [3]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0


In [4]:
!nvidia-smi

Thu Dec 23 04:53:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# **GIT 으로 NVCC Jupyter  다운로드**

In [6]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-c45xwxhy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-c45xwxhy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=1e043fa82f149a3f6e2a893527d06e04064284bc92ce02b54592c9d259d5db63
  Stored in directory: /tmp/pip-ephem-wheel-cache-01sb3icb/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [7]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:
%cd /usr/local

/usr/local


In [9]:
!pwd

/usr/local


In [10]:
!ls

bin	   cuda-11    games		  LICENSE.txt  setup.cfg
cuda	   cuda-11.0  _gcs_config_ops.so  licensing    share
cuda-10.0  cuda-11.1  include		  man	       src
cuda-10.1  etc	      lib		  sbin	       xgboost


# **CUDA 11버전 지우고 10버전 사용**

In [11]:
!rm -rf cuda

In [12]:
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [13]:
!stat cuda

  File: cuda -> /usr/local/cuda-10.1
  Size: 20        	Blocks: 0          IO Block: 4096   symbolic link
Device: 24h/36d	Inode: 4325382     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2021-12-23 04:53:53.521365179 +0000
Modify: 2021-12-23 04:53:49.425382333 +0000
Change: 2021-12-23 04:53:49.425382333 +0000
 Birth: -


# **예시 프로그램**

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8



In [ ]:
%%cu

# include <stdio.h>
 
__global__ void helloCUDA(void)
{
    printf("Hello CUDA from GPU! \n");
}
 
int main(void)
{
  helloCUDA<<<1, 1>>>();
  printf("Hello CUDA! \n");
  cudaDeviceReset();
  return 0;
}

Hello CUDA! 
Hello CUDA from GPU! 



In [ ]:
%%cu
#include <iostream>
int main(){
    std::cout << "hello world"<<std::endl;
}

hello world



In [ ]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// The size of the vector
#define NUM_DATA 512

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int *_a, int *_b, int *_c) {
	int tID = threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}

int main(void)
{

	int *a, *b, *c;	// Vectors on the host
	int *d_a, *d_b, *d_c;	// Vectors on the device

	int memSize = sizeof(int)*NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}

	// Memory allocation on the device-side
	cudaMalloc(&d_a, memSize);
	cudaMalloc(&d_b, memSize);
	cudaMalloc(&d_c, memSize);

	// Data copy : Host -> Device

	cudaMemcpy(d_a, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, memSize, cudaMemcpyHostToDevice);

	// Kernel call
	vecAdd<<<1, NUM_DATA >>>(d_a, d_b, d_c);

	// Copy results : Device -> Host
	cudaMemcpy(c, d_c, memSize, cudaMemcpyDeviceToHost);

	// Check results
	bool result = true;
	for (int i = 0; i < NUM_DATA; i++) {
		if (a[i] + b[i] != c[i]) {
			printf("[%d] The resutls is not matched! (%d, %d)\n"
				, i, a[i] + b[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release device memory
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}


512 elements, memSize = 2048 bytes
GPU works well!



DS Timer 다운받기
1. 다운받은 파일 usr/local/include 에 넣기
2. DS_Timer.cpp에 있는 거 없애고 .h에 다 넣기 (undefiend reference to - Error 발생)

In [ ]:
!git clone https://gitlab.com/DuksuKim/multi-core-programming-.-cph351-koreatech.git

Cloning into 'multi-core-programming-.-cph351-koreatech'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 531 (delta 104), reused 150 (delta 73), pack-reused 343
Receiving objects: 100% (531/531), 745.41 KiB | 11.47 MiB/s, done.
Resolving deltas: 100% (302/302), done.


# .cu 파일 작성 예시

In [ ]:
%%cuda --name my_curand.cu 
/*
 * This program uses the host CURAND API to generate 100 
 * pseudorandom floats.
 */
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <curand.h>

#define CUDA_CALL(x) do { if((x)!=cudaSuccess) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)
#define CURAND_CALL(x) do { if((x)!=CURAND_STATUS_SUCCESS) { \
    printf("Error at %s:%d\n",__FILE__,__LINE__);\
    return EXIT_FAILURE;}} while(0)

int main(int argc, char *argv[])
{
    size_t n = 100;
    size_t i;
    curandGenerator_t gen;
    float *devData, *hostData;

    /* Allocate n floats on host */
    hostData = (float *)calloc(n, sizeof(float));

    /* Allocate n floats on device */
    CUDA_CALL(cudaMalloc((void **)&devData, n*sizeof(float)));

    /* Create pseudo-random number generator */
    CURAND_CALL(curandCreateGenerator(&gen, 
                CURAND_RNG_PSEUDO_DEFAULT));

    /* Set seed */
    CURAND_CALL(curandSetPseudoRandomGeneratorSeed(gen, 
                1234ULL));

    /* Generate n floats on device */
    CURAND_CALL(curandGenerateUniform(gen, devData, n));

    /* Copy device memory to host */
    CUDA_CALL(cudaMemcpy(hostData, devData, n * sizeof(float),
        cudaMemcpyDeviceToHost));

    /* Show result */
    for(i = 0; i < n; i++) {
        printf("%1.4f ", hostData[i]);
    }
    printf("\n");

    /* Cleanup */
    CURAND_CALL(curandDestroyGenerator(gen));
    CUDA_CALL(cudaFree(devData));
    free(hostData);    
    return EXIT_SUCCESS;
}

'File written in /content/src/my_curand.cu'

cu 컴파일

In [ ]:
!nvcc -o /content/src/my_curand /content/src/my_curand.cu -lcurand

만든 파일 실행

In [ ]:
!/content/src/my_curand

0.1455 0.8202 0.5504 0.2948 0.9147 0.8690 0.3219 0.7829 0.0113 0.2855 0.7816 0.2338 0.6791 0.2824 0.6299 0.1212 0.4333 0.3831 0.5136 0.2987 0.4166 0.0345 0.0494 0.0467 0.6166 0.6480 0.8685 0.4012 0.0631 0.4972 0.6809 0.9350 0.0704 0.0458 0.1324 0.3785 0.6457 0.9930 0.9952 0.7677 0.3217 0.8210 0.2765 0.2691 0.4579 0.1969 0.9555 0.8739 0.7996 0.3810 0.6662 0.3153 0.9428 0.5006 0.3369 0.1490 0.8637 0.6191 0.6820 0.4573 0.9261 0.5650 0.7117 0.8252 0.8755 0.2216 0.2958 0.4046 0.3896 0.7335 0.7301 0.8154 0.0913 0.0866 0.6974 0.1811 0.5834 0.9255 0.9029 0.0413 0.9522 0.5507 0.7237 0.3976 0.7519 0.4398 0.4638 0.6094 0.7358 0.3272 0.6961 0.4893 0.9698 0.0456 0.2025 0.9491 0.1516 0.0424 0.6149 0.5638 


# Vector Sum 예제

In [ ]:
%%cuda --name vectorsum.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include "DS_timer.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// The size of the vector
#define NUM_DATA 10240

// Simple vector sum kernel (Max vector size : 1024)
__global__ void vecAdd(int *_a, int *_b, int *_c) {
	int tID = blockIdx.x*blockDim.x + threadIdx.x;
	_c[tID] = _a[tID] + _b[tID];
}


int main(void)
{
	// Set timer
	DS_timer timer(5);
	timer.setTimerName(0, "CUDA Total");
	timer.setTimerName(1, "Computation(Kernel)");
	timer.setTimerName(2, "Data Trans. : Host -> Device");
	timer.setTimerName(3, "Data Trans. : Device -> Host");
	timer.setTimerName(4, "VectorSum on Host");
	timer.initTimers();
	//timer.timerOff();

	int *a, *b, *c, *h_c;	// Vectors on the host
	int *d_a, *d_b, *d_c;	// Vectors on the device

	int memSize = sizeof(int)*NUM_DATA;
	printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

	// Memory allocation on the host-side
	a = new int[NUM_DATA]; memset(a, 0, memSize);
	b = new int[NUM_DATA]; memset(b, 0, memSize);
	c = new int[NUM_DATA]; memset(c, 0, memSize);
	h_c = new int[NUM_DATA]; memset(h_c, 0, memSize);

	// Data generation
	for (int i = 0; i < NUM_DATA; i++) {
		a[i] = rand() % 10;
		b[i] = rand() % 10;
	}

	// Vector sum on host (for performance comparision)
	timer.onTimer(4);
	for (int i = 0; i < NUM_DATA; i++)
		h_c[i] = a[i] + b[i];
	timer.offTimer(4);

	// Memory allocation on the device-side
	cudaMalloc(&d_a, memSize);
	cudaMalloc(&d_b, memSize);
	cudaMalloc(&d_c, memSize);

	timer.onTimer(0);

	// Data copy : Host -> Device
	timer.onTimer(2);
	cudaMemcpy(d_a, a, memSize, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, memSize, cudaMemcpyHostToDevice);
	timer.offTimer(2);

	// Kernel call
	timer.onTimer(1);

  dim3 gridSize(NUM_DATA/256,1,1);
  dim3 blockSize(256,1,1);
  vecAdd<<<gridSize, blockSize>>>(d_a,d_b,d_c);

	
	cudaDeviceSynchronize(); // synchronization function
	timer.offTimer(1);

	// Copy results : Device -> Host
	timer.onTimer(3);
	cudaMemcpy(c, d_c, memSize, cudaMemcpyDeviceToHost);
	timer.offTimer(3);

	timer.offTimer(0); timer.printTimer();

	// Check results
	bool result = true;
	for (int i = 0; i < NUM_DATA; i++) {
		if (h_c[i] != c[i]) {
			printf("[%d] The resutls is not matched! (%d, %d)\n"
				, i, h_c[i], c[i]);
			result = false;
		}
	}

	if (result)
		printf("GPU works well!\n");

	// Release device memory
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	// Release host memory
	delete[] a; delete[] b; delete[] c;

	return 0;
}


'File written in /content/src/vectorsum.cu'

In [ ]:
!nvcc -o /content/src/vectorsum /content/src/vectorsum.cu -lcurand

/content/src/vectorsum.cu(24): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(25): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(26): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(27): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(28): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(24): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(25): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(26): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(27): warning: conversion from a string literal to "char *" is deprecated

/content/src/vectorsum.cu(28): warning: conversion from a string literal to "char *" is deprecated



In [ ]:
!/content/src/vectorsum

10240 elements, memSize = 40960 bytes

*	 DS_timer Report 	*
* The number of timer = 5, counter = 5
**** Timer report ****
CUDA Total : 0.23900 ms (0.23900 ms)
Computation(Kernel) : 0.13000 ms (0.13000 ms)
Data Trans. : Host -> Device : 0.07800 ms (0.07800 ms)
Data Trans. : Device -> Host : 0.03100 ms (0.03100 ms)
VectorSum on Host : 0.03200 ms (0.03200 ms)
**** Counter report ****
*	 End of the report 	*
GPU works well!


# Matrix Multiply_Single 예제

In [ ]:
%%cuda --name MatrixMultiply.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <DS_timer.h>

#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define NUM_CPU_THREADS 4

#define ROW_SIZE 32
#define K_SIZE 128
#define COL_SIZE 32

#define WORK_LOAD  1024
#define WORK_LOAD2  100
#define MAT_SIZE_A ROW_SIZE*K_SIZE
#define MAT_SIZE_B K_SIZE*COL_SIZE
#define MAT_SIZE_C ROW_SIZE*COL_SIZE

// input matrix
float A[ROW_SIZE][K_SIZE];     // m*k
float B[K_SIZE][COL_SIZE];     // k*n

// timer
DS_timer* timer;
#define TIMER_HOST		0
#define TIMER_KERNEL	1
#define TIMER_KERNEL_NC	2
#define TIMER_KERNEL_SH	3
#define TIMER_KERNEL_SH_BC	4
#define TIMER_HtoD		5
#define TIMER_DtoH		6
#define NUM_TIMER		(TIMER_DtoH+1)

void setTimer(void);
void genInputMatrix(void);

// output matrix
float hostC[ROW_SIZE][COL_SIZE];
float deviceC[COL_SIZE][COL_SIZE];

#define memsetZero(_P,_type,_size) memset(_P, 0, sizeof(_type)*_size);
#define dMemAlloc(_P, _type, _size) cudaMalloc(&_P, sizeof(_type)*_size);

__global__ void matMul_kernel(float* _A, float* _B, float* _C){
    int row = threadIdx.y;
    int col = threadIdx.x;
    int index = row*blockDim.x + col;

    _C[index] = 0;
    for(int k=0;k<K_SIZE;k++)
        for(int i=0;i<WORK_LOAD;i++)
            for(int j=0;j<WORK_LOAD2;j++)
                _C[index] += _A[row*K_SIZE + k]*_B[k*COL_SIZE + col];
}

__global__ void matMul_kernel_NC(float* _A, float* _B, float* _C){
    int row = threadIdx.x;
    int col = threadIdx.y;
    int index = row*blockDim.y + col;

    _C[index] = 0;
    for(int k=0;k<K_SIZE;k++)
        for(int i=0;i<WORK_LOAD;i++)
            for(int j=0;j<WORK_LOAD2;j++)
                _C[index] += _A[row*K_SIZE + k]*_B[k*COL_SIZE + col];
}

__global__ void matMul_kernel_shared(float* _A, float* _B, float* _C){
    int row = threadIdx.y;
    int col = threadIdx.x;
    int index = row*blockDim.x + col;

    __shared__ float sharedA[ROW_SIZE][K_SIZE]; // Shared Memory에 공간 잡기
    __shared__ float sharedB[K_SIZE][COL_SIZE];

    for(int k=0;k<K_SIZE;k++){
        sharedA[row][k] = _A[row*K_SIZE + k];
        sharedB[k][col] = _B[k*COL_SIZE + col];
    }
    __syncthreads(); // Shared Memory에 모든 값이 다 올라갈 때 까지 기다림 

    _C[index] = 0;
    for(int k=0;k<K_SIZE;k++)
        for(int i=0;i<WORK_LOAD;i++)
            for(int j=0;j<WORK_LOAD2;j++)
                _C[index] += sharedA[row][k] * sharedB[k][col];
}

__global__ void matMul_kernel_shared_BC(float* _A, float* _B, float* _C){
    int row = threadIdx.x;
    int col = threadIdx.y;
    int index = row*blockDim.y + col;

    __shared__ float sharedA[ROW_SIZE][K_SIZE]; // Shared Memory에 공간 잡기
    __shared__ float sharedB[K_SIZE][COL_SIZE];

    for(int k=0;k<K_SIZE;k++){
        sharedA[row][k] = _A[row*K_SIZE + k];
        sharedB[k][col] = _B[k*COL_SIZE + col];
    }
    __syncthreads(); // Shared Memory에 모든 값이 다 올라갈 때 까지 기다림 

    _C[index] = 0;
    for(int k=0;k<K_SIZE;k++)
        for(int i=0;i<WORK_LOAD;i++)
            for(int j=0;j<WORK_LOAD2;j++)
                _C[index] += sharedA[row][k] * sharedB[k][col];
}

int main(void){
    timer = NULL; setTimer();
    float *dA, *dB, *dC; 
    dA= dB = dC = NULL;
    
    memsetZero(A,float,MAT_SIZE_A); memsetZero(B,float,MAT_SIZE_B);
    memsetZero(hostC,float,MAT_SIZE_C); memsetZero(deviceC,float,MAT_SIZE_C);

    // device(GPU) memory Allocation
    dMemAlloc(dA,float,MAT_SIZE_A);
    dMemAlloc(dB,float,MAT_SIZE_B);
    dMemAlloc(dC,float,MAT_SIZE_C);

    genInputMatrix();

    //Host Code
    timer->onTimer(TIMER_HOST);
    for(int r=0;r<ROW_SIZE;r++)
        for(int c=0;c<COL_SIZE;c++)
            for(int k=0;k<K_SIZE;k++)
                for(int i=0;i<WORK_LOAD;i++)
                    for(int j=0;j<WORK_LOAD2;j++)
                        hostC[r][c] += A[r][k] * B[k][c];
    timer->offTimer(TIMER_HOST);
    // ------------------------ Device(GPU)-------------------------//
    //Copy input matrics : Host To Device (CPU > GPU)
    timer->onTimer(TIMER_HtoD);
    cudaMemcpy(dA,A,sizeof(float)*MAT_SIZE_A,cudaMemcpyHostToDevice);
    cudaMemcpy(dB,B,sizeof(float)*MAT_SIZE_B,cudaMemcpyHostToDevice);
    timer->offTimer(TIMER_HtoD);

    dim3 blockDim(COL_SIZE,ROW_SIZE);     // 2D Block Dimension
    dim3 blockDim_NC(ROW_SIZE,COL_SIZE);  // 2D Block Dimension Not Coalesced
    
    // Mat Multiply kernel call
    timer->onTimer(TIMER_KERNEL);
    matMul_kernel <<<1,blockDim>>> (dA,dB,dC);
    cudaThreadSynchronize();
    timer->offTimer(TIMER_KERNEL);

    // Mat Multiply Not Coalesced kernel call
    timer->onTimer(TIMER_KERNEL_NC);
    matMul_kernel_NC <<<1,blockDim_NC>>> (dA,dB,dC);
    cudaThreadSynchronize();
    timer->offTimer(TIMER_KERNEL_NC);

    // Shared Mem Mat Multiply kernel call
    timer->onTimer(TIMER_KERNEL_SH);
    matMul_kernel_shared <<<1,blockDim>>> (dA,dB,dC);
    cudaThreadSynchronize();
    timer->offTimer(TIMER_KERNEL_SH);

    // Shared Mem Mat Multiply kernel call has Bank Conflict
    timer->onTimer(TIMER_KERNEL_SH_BC);
    matMul_kernel_shared_BC <<<1,blockDim_NC>>> (dA,dB,dC);
    cudaThreadSynchronize();
    timer->offTimer(TIMER_KERNEL_SH_BC);

    // Get Result From Device : Device To Host (GPU > CPU)
    timer->onTimer(TIMER_DtoH);
    cudaMemcpy(deviceC,dC,sizeof(float)*MAT_SIZE_C,cudaMemcpyDeviceToHost);
    timer->offTimer(TIMER_DtoH);

    // Check Result 
    bool isSame = true;

    float *pHostC = &hostC[0][0];
    float *pDeviceC = &deviceC[0][0];

    for(int i=0;i<MAT_SIZE_C;i++){
        if(pHostC[i] != pDeviceC[i]){
            printf("[%d] %.2f, %.2f\n", i, pHostC[i],pDeviceC[i]);
            isSame = false;
            break;
        }
    }

    if(isSame) printf("Result is Same On CPU & GPU! \n");
    else printf("Result is Different on CPU & GPU. Something is Wrong! \n");

    timer->printTimer();
    if(timer != NULL)
        delete timer;
}

void genInputMatrix(void){
    for(int r=0;r<ROW_SIZE;r++)
        for(int k=0;k<K_SIZE;k++)
            A[r][k] = rand() % 100;
    
    for(int k=0;k<K_SIZE;k++)
        for(int c=0;c<COL_SIZE;c++)
            B[k][c] = rand() % 100;
}

void setTimer(void){
    timer = new DS_timer(NUM_TIMER);

    timer->initTimers();
    timer->setTimerName(TIMER_HOST," CPU code                    ");
    timer->setTimerName(TIMER_KERNEL," Kernel launch(Coalasced)    ");
    timer->setTimerName(TIMER_KERNEL_NC," Kernel launch(Not Coalasced)");
    timer->setTimerName(TIMER_KERNEL_SH," Kernel launch (shared ver.) ");
    timer->setTimerName(TIMER_KERNEL_SH_BC,"Kernel launch (shared ver)BC ");
    timer->setTimerName(TIMER_HtoD,"[Data Transfer] Host > Device");
    timer->setTimerName(TIMER_DtoH,"[Data Transfer] Device > Host");
}

'File written in /content/src/MatrixMultiply.cu'

In [ ]:
!nvcc -o /content/src/MatrixMultiply /content/src/MatrixMultiply.cu -lcurand

/content/src/MatrixMultiply.cu(213): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(214): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(215): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(216): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(217): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(218): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(219): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(213): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(214): warning: conversion from a string literal to "char *" is deprecated

/content/src/MatrixMultiply.cu(215): warning: 

In [ ]:
!/content/src/MatrixMultiply

Result is Same On CPU & GPU! 

*	 DS_timer Report 	*
* The number of timer = 7, counter = 7
**** Timer report ****
 CPU code                     : 57163.31900 ms (57163.31900 ms)
 Kernel launch(Coalasced)     : 3950.27400 ms (3950.27400 ms)
 Kernel launch(Not Coalasced) : 52928.72900 ms (52928.72900 ms)
 Kernel launch (shared ver.)  : 141.90300 ms (141.90300 ms)
Kernel launch (shared ver)BC  : 142.19600 ms (142.19600 ms)
[Data Transfer] Host > Device : 0.08900 ms (0.08900 ms)
[Data Transfer] Device > Host : 0.12600 ms (0.12600 ms)
**** Counter report ****
*	 End of the report 	*
